# レスラー方程式(ランダムなシフト付きsin外力のある状態)

レスラー方程式の外力のある場合に関して，$\sin$ 波に位相のランダムなシフトがある場合を考える．

In [21]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os

rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


In [22]:
def phase_shift(t, shift_hour):
    # 2πは1日に対応します。8時間は(8/24)*2πに対応します。
    shift_value = (shift_hour/24)*2*np.pi
    # 4日ごとに位相をシフトします。
    number_of_shifts = int(t / (4*2*np.pi))
    return number_of_shifts * shift_value


In [23]:
# 位相のシフトする時間を定める．
shift_hours = range(-12, 13)


In [24]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def coupled_system(t, state, mu, sigma, rho, beta, A, shift_hour):
    X, Y= state

    dXdt = Y
    dYdt = mu * (1 - X**2) * Y - X +  A * np.sin(t + phase_shift(t, shift_hour)) 
    
    return [dXdt, dYdt]


In [25]:

mu = 5.0
sigma = -0.1
rho = 28.0
beta = 8/3
A = 5.0
initial_state = [1.0, 0.0]  # Initial states for [X, Y, x_lorenz, y_lorenz, z_lorenz]

# 時間の設定
t_span = [0, 10000]  # 開始時刻と終了時刻
t_eval = np.linspace(t_span[0], t_span[1], 100000)  # 評価する時間点

In [26]:
for shift_hour in shift_hours:
    # 新しいディレクトリを作成
    dir_name = f"data_definite"
    os.makedirs(dir_name, exist_ok=True)
    
    sol = solve_ivp(coupled_system, t_span, initial_state, args=(mu, sigma, rho, beta, A, shift_hour), t_eval=t_eval, max_step = 0.01)

    # 外力P(t)の計算（位相シフト付き）
    P_shifted = A * np.sin(sol.t + np.array([phase_shift(ti, shift_hour) for ti in sol.t]))

    # DataFrameの作成
    data_frame_with_shifted_force = pd.DataFrame({
        'time': sol.t,
        'x': sol.y[0],
        'y': sol.y[1],
        'P_shifted': P_shifted# 位相シフトされた外力P(t)の列を追加
    })

    filename_with_force = f"{dir_name}/VDP_{shift_hour}.csv"

    # CSVファイルにデータを保存
    data_frame_with_shifted_force.to_csv(filename_with_force, index=False)